In [18]:
import pandas as pd

df = pd.read_csv("/content/marathon.csv")
df.head(1)

,Rank,Time,Name,Country,Date of Birth,Place,City,Date,Gender,Event
0,1,2:00:35,Kelvin Kiptum,KEN,02.12.1999,1,Chicago,08.10.2023,Men,Marathon


In [ ]:
import pandas as pd
from datetime import datetime
import io

import warnings
warnings.filterwarnings('ignore')

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')

# Sort by Gender and Date to get chronological order of records
df_sorted = df.sort_values(['Gender', 'Date'])

# Calculate record duration for each athlete
def calculate_record_duration(group):
    # Sort by date within each gender group
    group = group.sort_values('Date').copy()

    # Calculate duration each record was held
    group['Record_End_Date'] = group['Date'].shift(-1)

    # For the last (current) record holder, use today's date
    today = datetime.now()
    group['Record_End_Date'] = group['Record_End_Date'].fillna(today)

    # Calculate duration in days
    group['Duration_Days'] = (group['Record_End_Date'] - group['Date']).dt.days

    return group

# Apply the function to each gender group
df_with_duration = df_sorted.groupby('Gender').apply(calculate_record_duration).reset_index(drop=True)

for gender in df_with_duration['Gender'].unique():
    gender_data = df_with_duration[df_with_duration['Gender'] == gender].copy()

    print(f"\n{gender.upper()} MARATHON RECORDS:")
    print("-" * 60)

    # Sort by duration (descending) and take top 20
    gender_data_sorted = gender_data.sort_values('Duration_Days', ascending=False).head(20)

    for idx, row in gender_data_sorted.iterrows():
        end_status = "Present" if pd.isna(df_sorted[df_sorted['Gender'] == gender]['Date'].shift(-1).iloc[0] if len(df_sorted[df_sorted['Gender'] == gender]) == 1 else row['Record_End_Date']) or row['Record_End_Date'].date() == datetime.now().date() else row['Record_End_Date'].strftime('%Y-%m-%d')

        print(f"{row['Name']:<20} | {row['Time']:<8} | {row['Date'].strftime('%Y-%m-%d')} | {row['Duration_Days']:>4} days")


MEN MARATHON RECORDS:
------------------------------------------------------------
Yuriy Volkov         | 2:14:28  | 1970-12-06 |  509 days
Hidekuni Hiroshima   | 2:14:06  | 1966-11-27 |  371 days
Brian Kilby          | 2:14:43  | 1963-07-06 |  343 days
Toru Terasawa        | 2:13:41  | 1965-06-12 |  246 days
Abebe Bikila         | 2:12:12  | 1964-10-21 |  234 days
Ferdie La Grange     | 2:12:47  | 1974-04-23 |  229 days
Gerald Umbach        | 2:14:43  | 1977-04-30 |  218 days
Ryo Matsumoto        | 2:10:32  | 2020-03-08 |  210 days
Gyula Tóth           | 2:14:59  | 1968-05-19 |  203 days
Derek Clayton        | 2:08:34  | 1969-05-30 |  191 days
Bernd Amhold         | 2:14:12  | 1975-06-15 |  175 days
Dave Mckenzie        | 2:12:26  | 1967-12-03 |  168 days
Hirokazu Okabe       | 2:14:52  | 1966-02-13 |  146 days
Alastair Wood        | 2:13:45  | 1966-07-09 |  141 days
Inocencio Miranda    | 2:13:40  | 1995-05-07 |  140 days
Reino Paukkonen      | 2:14:15  | 1973-09-15 |  138 days
Jeff

In [24]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("/content/marathon.csv")
df = df[df['Country'] == 'RUS']

# Convert Date column to datetime and extract year
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y')
df['Event_Year'] = df['Date'].dt.year

# Group athletes by Country, Event, and Event_Year to find potential training groups
training_groups = df.groupby(['Country', 'Event_Year']).agg({
    'Name': list,
    'City': lambda x: list(set(x)),  # Get unique cities
    'Date': lambda x: list(set(x.dt.strftime('%d.%m.%Y'))),  # Get unique dates
    'Gender': list,
    'Time': list
}).reset_index()

# Add group size
training_groups['Group_Size'] = training_groups['Name'].apply(len)

# Filter for groups with more than 1 athlete (potential training partners)
potential_training_groups = training_groups[training_groups['Group_Size'] > 1]

print("POTENTIAL TRAINING GROUPS:")
print("Athletes from the same country competing during the same year")
print("="*80)

if not potential_training_groups.empty:
    for idx, group in potential_training_groups.iterrows():
        print(f"\nTraining Group #{idx + 1}:")
        print(f"Country: {group['Country']}")
        print(f"Year: {group['Event_Year']}")
        print(f"Group Size: {group['Group_Size']} athletes")
        print(f"Cities competed in: {', '.join(group['City'])}")
        print(f"Competition dates: {', '.join(group['Date'])}")
        print("Athletes:")
        for i, (name, gender, time) in enumerate(zip(group['Name'], group['Gender'], group['Time']), 1):
            print(f"  {i}. {name} ({gender}) - {time}")
        print("-" * 50)
else:
    print("No training groups found with the current criteria.")

print("\n" + "="*80)
print("DETAILED ANALYSIS BY COUNTRY AND EVENT:")
print("="*80)

# More detailed analysis - show all athletes grouped by country and event
detailed_groups = df.groupby(['Country', 'Event']).agg({
    'Name': list,
    'Event_Year': list,
    'City': list,
    'Date': lambda x: list(x.dt.strftime('%d.%m.%Y')),
    'Gender': list,
    'Time': list
}).reset_index()

for idx, group in detailed_groups.iterrows():
    print(f"\n{group['Country']} - {group['Event']}:")
    years = list(set(group['Event_Year']))
    print(f"Active years: {', '.join(map(str, sorted(years)))}")
    print(f"Total athletes: {len(group['Name'])}")
    print("Athletes details:")
    for name, year, city, date, gender, time in zip(
        group['Name'], group['Event_Year'], group['City'],
        group['Date'], group['Gender'], group['Time']
    ):
        print(f"  • {name} ({gender}) - {time} in {city} on {date} ({year})")
    print("-" * 50)

# Additional analysis: Same year, same country clusters
print("\n" + "="*80)
print("SAME YEAR CLUSTERS (Regardless of exact date):")
print("="*80)

same_year_groups = df.groupby(['Country', 'Event_Year']).agg({
    'Name': list,
    'Event': list,
    'City': list,
    'Date': lambda x: list(x.dt.strftime('%d.%m.%Y')),
    'Gender': list,
    'Time': list
}).reset_index()

same_year_groups['Group_Size'] = same_year_groups['Name'].apply(len)
multi_athlete_years = same_year_groups[same_year_groups['Group_Size'] > 1]

if not multi_athlete_years.empty:
    for idx, group in multi_athlete_years.iterrows():
        print(f"\n{group['Country']} - {group['Event_Year']}:")
        print(f"Athletes: {group['Group_Size']}")
        events = list(set(group['Event']))
        print(f"Events: {', '.join(events)}")
        for name, event, city, date, gender, time in zip(
            group['Name'], group['Event'], group['City'],
            group['Date'], group['Gender'], group['Time']
        ):
            print(f"  • {name} ({gender}) - {event} - {time} in {city} on {date}")
        print("-" * 50)
else:
    print("No multi-athlete clusters found in the same year.")

# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS:")
print("="*80)
print(f"Total athletes: {len(df)}")
print(f"Countries represented: {df['Country'].nunique()}")
print(f"Events: {', '.join(df['Event'].unique())}")
print(f"Years covered: {df['Event_Year'].min()} - {df['Event_Year'].max()}")
print(f"Potential training groups (same country, event, year): {len(potential_training_groups)}")

POTENTIAL TRAINING GROUPS:
Athletes from the same country competing during the same year

Training Group #1:
Country: RUS
Year: 1992
Group Size: 3 athletes
Cities competed in: Paris, Cesano Boscone, Rouen
Competition dates: 29.03.1992, 08.11.1992, 18.10.1992
Athletes:
  1. Tatyana Titova (Women) - 2:31:12
  2. Fyodor Ryzhov (Men) - 2:13:02
  3. Ruslan Yamayev (Men) - 2:14:50
--------------------------------------------------

Training Group #2:
Country: RUS
Year: 1993
Group Size: 5 athletes
Cities competed in: Paris, Kaliningrad, Fukuoka, San Sebastián
Competition dates: 31.10.1993, 05.12.1993, 25.04.1993
Athletes:
  1. Aleksey Zhelonkin (Men) - 2:10:44
  2. Tatyana Ivanova-Zolotareva (Women) - 2:33:52
  3. Olga Michurina (Women) - 2:34:31
  4. Andrey Tarasov (Men) - 2:12:56
  5. Nikolay Plykin (Men) - 2:14:53
--------------------------------------------------

Training Group #3:
Country: RUS
Year: 1994
Group Size: 5 athletes
Cities competed in: Paris, Osaka, Kaliningrad, Echternach
Co